# OpenStreetMap extracts

**QuackOSM** exposes a simple API allowing user to download OSM extracts from multiple sources.

This example notebook shows how to: 
 - display available extracts,
 - query extracts by name,
 - find extracts covering a given geometry.

<style>
div.jp-Cell-outputArea pre {
  overflow-y: auto;
  max-height: 50vh;
}
</style>

## Display available OSM extracts

Extracts have short name and a full name. Short name is just a current description of the region (eg. `Monaco`). Full name contains information about the extract source and all a whole nesting hierarchy (eg. `geofabrik_europe_monaco`).

By default, function for displaying those extracts shows short name. In these examples below full names have been used.

In [ ]:
from quackosm.osm_extracts import display_available_extracts, OsmExtractSource

### Geofabrik

In [ ]:
display_available_extracts(OsmExtractSource.geofabrik)

### OpenStreetMap FR

In [ ]:
display_available_extracts(OsmExtractSource.osm_fr)

### BBBike

In [ ]:
display_available_extracts(OsmExtractSource.bbbike)

## Query OSM extract by name

In [ ]:
from quackosm import convert_osm_extract_to_geodataframe

### Vatican city

Download data for the Vatican city from any repository. Only `OpenStreetMap.fr` contains data for this query.

In [ ]:
convert_osm_extract_to_geodataframe("Vatican City")

### Monaco extract from Geofabrik

Download data for the country of Monaco from the `Geofabrik` repository.

In [ ]:
convert_osm_extract_to_geodataframe("Monaco", osm_extract_source="Geofabrik")

### Query with multiple matches

Some extracts have the same name, or the same extract is available in multiple sources. Trying to get an extract by name with multiple matches will result in an error with list of extracts ids that can be used to get a specific one.

Getting a **Ceuta** region (autonomous city of Spain bordering with Marocco).

Extract for this region is available both in Geofabrik and OSM fr sources.

In [ ]:
from rich import print as rprint
from rich.traceback import Traceback

from quackosm._exceptions import OsmExtractMultipleMatchesError

try:
    convert_osm_extract_to_geodataframe("Ceuta")
except OsmExtractMultipleMatchesError as ex:
    rprint(Traceback.from_exception(type(ex), ex, None))

`OsmExtractMultipleMatchesError` has a property `matching_full_names` with a list of found extracts full names. It can be used programatically to access those values.

In [ ]:
from quackosm.osm_extracts import get_extract_by_query

matched_extracts = []

try:
    get_extract_by_query("Ceuta")
except OsmExtractMultipleMatchesError as ex:
    for full_name in ex.matching_full_names:
        matched_extracts.append(get_extract_by_query(full_name))

matched_extracts

We can display both extracts extents on the map.

In [ ]:
from quackosm.osm_extracts.extract import extracts_to_geodataframe

extracts_to_geodataframe(matched_extracts).explore(column="id", tiles="CartoDB positron")

Let's download data for the extract from the OpenStreetMap.fr.

In [ ]:
convert_osm_extract_to_geodataframe("osmfr_africa_spain_ceuta")

### Query with zero matches

Sometimes query doesn't match any of the available extracts.
Function for finding the extracts, automatically detects close names in case this was a typo and suggests them to the user. Suggestions are sorted based on the closeness to the query.

In [ ]:
from quackosm._exceptions import OsmExtractZeroMatchesError

try:
    convert_osm_extract_to_geodataframe("Gremany")
except OsmExtractZeroMatchesError as ex:
    rprint(Traceback.from_exception(type(ex), ex, None))

`OsmExtractZeroMatchesError` has a property `matching_full_names` with a list of suggested matching names. It can be used programatically to access those values.

In [ ]:
closest_matching_extract = None

try:
    get_extract_by_query("pland")
except OsmExtractZeroMatchesError as ex:
    rprint(Traceback.from_exception(type(ex), ex, None))
    closest_matching_extract_name = ex.matching_full_names[0]
    closest_matching_extract = get_extract_by_query(closest_matching_extract_name)

closest_matching_extract

Sometimes query can yield zero closest matches.

In [ ]:
try:
    get_extract_by_query("totally_nonexistent_osm_extract")
except OsmExtractZeroMatchesError as ex:
    rprint(Traceback.from_exception(type(ex), ex, None))

## Find OSM extracts covering a given geometry

One ot the most interesting feature of **QuackOSM** is the ability to automatically find extracts for a selected region without any domain knowledge of the services providing these extracts.

Search algorithm tries to find the best matching extracts to cover a given geometry filter.

<style>
  .jp-Mermaid {
    display: block !important;
  }
  .mermaid {
    display: flex;
    justify-content: center;
  }
</style>

### Flowchart

Here is the flowchart diagram of the algorithm:

```mermaid
flowchart TD
    A(Input Geometry)
    B{"Geometry
        type?"}
    B1[Split to Polygons]

    subgraph 1["For each Polygon (in a loop)"]
        D["Intersect Polygon
            with OSM extracts"]
        E{"Number of
            matching
            extracts?"}
        E1{"`allow
            uncovered
            geometry`"}
        E3[Warn User]
        F["Calculate IoU
            between extracts
            and a Polygon"]
        G["Select extract
            with the highest
            IoU value"]
        H{"IoU >= threshold"}
        H1[Keep the extract]
        H2[Discard the extract]
        I["Remove the intersection
            area from the Polygon"]
        J{Is Polygon empty?}
        K(Exit loop)
    end

    E2(Raise Error)

    L["Sort selected
        extracts by area
        descending"]

    subgraph 2["Filter selected extracts (for each Polygon in a loop)"]
        M["Initialize empty list
            of filtered extracts"]
        N{"Is next extract?"}
        N1["Select next extract"]
        N2("Exit loop")
        O{"Is current extract
            disjoint with Polygon?"}
        O1["Keep the extract"]
        O2["Discard the extract"]
        P["Remove the intersection
            area from the Polygon"]
    end

    Q["Join lists of
        filtered extracts"]

    subgraph 3["Simplify filtered extracts"]
        R["Start iterating filtered extracts"]
        S{"Is next extract?"}
        S1["Select next extract"]
        S2("Exit loop")
        T["Union geometries
            of every other extract"]
        U{"Is current
            extract covered
            by other extracts?"}
        U1["Discard the extract"]
        U2["Keep the extract"]
    end

    V("Return simplified extracts")



    A --> B
    B -- MultiPolygon --> B1
    B -- Polygon --> D
    B1 --> D
    D --> E
    E -- 0 --> E1
    E1 -- true --> E3
    E1 -- false ----> E2
    E -- > 0 --> F
    F --> G
    G --> H
    H -- Yes --> H1
    H -- No --> H2
    H1 --> I
    H2 --> I
    I --> J
    J -- Yes --> K
    J -- No --> D
    E3 --> K
    K --> L
    L --> M
    M --> N
    N -- Yes --> N1
    N -- No ------> N2
    N1 --> O
    O -- Yes --> O2
    O -- No --> O1
    O1 --> P
    O2 --> N
    P --> N
    
    N2 --> Q
    Q --> R
    R --> S
    S -- Yes --> S1
    S -- No -----> S2
    S1 --> T
    T --> U
    U -- Yes --> U1
    U -- No --> U2
    U1 --> R
    U2 --> S
    S2 --> V
  
```

### Bounding box within the city example

In [ ]:
from quackosm.osm_extracts import _cover_geometry_with_extracts, _get_combined_index, _get_geofabrik_index
from quackosm.geocode import geocode_to_geometry
from quackosm.osm_extracts import find_smallest_containing_extracts_total, find_smallest_containing_geofabrik_extracts
from shapely import from_wkt
from shapely.geometry import shape

In [ ]:
from typing import Optional
from matplotlib import pyplot as plt
import contextily as cx
import geopandas as gpd
from shapely import Polygon

def plot_full_geometry_coverage_breakdown(geometry_filter: Polygon, index: Optional[gpd.GeoDataFrame] = None) -> None:
    if index is None:
        index = _get_combined_index()
    extract_ids, iou_metrics = _cover_geometry_with_extracts(geometry_filter, index)
    geometry_to_cover = geometry_filter.buffer(0)

    total_extracts = len(extract_ids)
    fig, axes = plt.subplots(nrows=total_extracts, ncols=4, figsize=(20, 5 * total_extracts))

    close_up_bbox = geometry_to_cover.bounds
    full_bbox = index[index["id"].isin(extract_ids)].union_all().bounds

    for ax_idx, (extract_id, iou_metric) in enumerate(zip(extract_ids, iou_metrics)):
        iou_metric_above_threshold = iou_metric > 0.01

        if total_extracts > 1:
            combined_ax = axes[ax_idx, 0]
            geometry_to_cover_ax = axes[ax_idx, 1]
            intersection_geometry_ax = axes[ax_idx, 2]
            extract_geometry_ax = axes[ax_idx, 3]
        else:
            combined_ax = axes[0]
            geometry_to_cover_ax = axes[1]
            intersection_geometry_ax = axes[2]
            extract_geometry_ax = axes[3]

        combined_ax.set_xlim([full_bbox[0], full_bbox[2]])
        combined_ax.set_ylim([full_bbox[1], full_bbox[3]])

        combined_ax.set_yticklabels([])
        combined_ax.set_xticklabels([])
        combined_ax.set_xticks([])
        combined_ax.set_yticks([])

        cx.add_basemap(combined_ax, source=cx.providers.CartoDB.Positron, crs=4326)

        for ax in (geometry_to_cover_ax, intersection_geometry_ax, extract_geometry_ax):
            ax.set_xlim([close_up_bbox[0], close_up_bbox[2]])
            ax.set_ylim([close_up_bbox[1], close_up_bbox[3]])

            ax.set_axis_off()

            cx.add_basemap(ax, source=cx.providers.CartoDB.PositronNoLabels, crs=4326)

        extract_row = index[index["id"] == extract_id].iloc[0]
        combined_ax.set_title(extract_row["file_name"])
        if ax_idx == 0:
            combined_ax.set_ylabel(f"Step {ax_idx + 1} (always accepted)")
        else:
            combined_ax.set_ylabel(
                f'Step {ax_idx + 1} ({"accepted" if iou_metric_above_threshold else "discarded"})'
            )

        geometry_to_cover_ax.set_title("Geometry to cover")
        extract_geometry_ax.set_title("Extract geometry close-up")
        intersection_geometry_ax.set_title(f"Intersection (IoU: {iou_metric:.3g})")

        extract_geometry = extract_row["geometry"]
        intersection_geometry = extract_geometry.intersection(geometry_to_cover)
        gpd.GeoSeries([geometry_to_cover], crs=4326).plot(
            ax=geometry_to_cover_ax, color="C0", alpha=0.8
        )
        gpd.GeoSeries([geometry_to_cover], crs=4326).plot(ax=combined_ax, color="C0", alpha=0.8)

        gpd.GeoSeries([extract_geometry], crs=4326).plot(
            ax=combined_ax,
            color="C2" if (iou_metric_above_threshold or ax_idx == 0) else "C3",
            alpha=0.4,
            zorder=1,
        )
        gpd.GeoSeries([extract_geometry], crs=4326).plot(
            ax=extract_geometry_ax,
            color="C2" if (iou_metric_above_threshold or ax_idx == 0) else "C3",
            alpha=0.4,
            zorder=1,
        )

        gpd.GeoSeries([intersection_geometry], crs=4326).plot(
            ax=combined_ax,
            color=(0, 0, 0, 0),
            zorder=2,
            hatch="///",
            edgecolor="C1",
            linewidth=1.5,
        )

        gpd.GeoSeries([intersection_geometry], crs=4326).plot(
            ax=intersection_geometry_ax,
            color=(0, 0, 0, 0),
            zorder=2,
            hatch="///",
            edgecolor="C1",
            linewidth=1.5,
        )

        geometry_to_cover = geometry_to_cover.difference(intersection_geometry)

    fig.tight_layout()

    plt.show()

In [ ]:
geometry_filter = geocode_to_geometry("Monaco-Ville, Monaco")
matched_extracts = find_smallest_containing_extracts_total(geometry_filter)
plot_full_geometry_coverage_breakdown(geometry_filter)
matched_extracts

In [ ]:
geometry_filter = geocode_to_geometry("Monaco")
matched_extracts = find_smallest_containing_geofabrik_extracts(geometry_filter)
plot_full_geometry_coverage_breakdown(geometry_filter, _get_geofabrik_index())
matched_extracts

In [ ]:
geometry_filter = from_wkt(
    "POLYGON ((1.382599544073372 42.67676873293743, 1.382599544073372 42.40065303248514,"
    " 1.8092269635579328 42.40065303248514, 1.8092269635579328 42.67676873293743,"
    " 1.382599544073372 42.67676873293743))"
)
matched_extracts = find_smallest_containing_extracts_total(geometry_filter)
plot_full_geometry_coverage_breakdown(geometry_filter)
matched_extracts

In [ ]:
geometry_filter = shape(
    {
        "type": "Polygon",
        "coordinates": [
            [
                [-122.04132711751888, 39.47952317060893],
                [-122.04132711751888, 38.586779237435586],
                [-121.15664232594925, 38.586779237435586],
                [-121.15664232594925, 39.47952317060893],
                [-122.04132711751888, 39.47952317060893],
            ]
        ],
    }
)
matched_extracts = find_smallest_containing_extracts_total(geometry_filter)
plot_full_geometry_coverage_breakdown(geometry_filter)
matched_extracts